In [ ]:
# Aim is to check how well European options can be replicated by trading

In [61]:
import numpy as np
import pandas as pd

# Preprocess approach 1
data = pd.read_csv('../data/csco_2025.csv')

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

prices = data['Close/Last'].values
prices = prices[::-1]

print(prices)

n = prices.size
delta_t = 1 / n

start = n // 2
i = np.arange(0, start)
X = np.log(prices[i+1] / prices[i])

sigma = np.std(X) / np.sqrt(delta_t)
mu = np.mean(X) / delta_t + sigma**2 / 2

print("Mu:", mu)
print("Sigma:", sigma)

[48.95 49.67 48.34 48.17 47.13 46.94 47.43 46.6  46.42 46.28 46.08 46.12
 46.5  46.65 47.42 46.02 46.23 45.84 45.85 45.77 45.62 45.52 45.68 45.69
 45.97 46.72 47.29 47.28 47.38 46.86 47.45 47.51 47.52 47.28 47.09 46.65
 46.06 45.8  46.27 46.76 47.38 47.4  47.42 48.52 48.04 47.32 46.82 46.36
 46.85 47.24 47.88 47.99 48.14 48.45 47.39 46.66 44.74 45.16 45.12 45.83
 45.47 44.86 45.37 45.44 48.53 49.46 49.98 50.24 50.45 50.22 50.74 50.79
 50.71 49.85 50.39 50.54 50.06 49.7  49.13 48.5  48.7  48.79 49.11 49.61
 49.8  51.03 50.61 50.57 51.44 51.97 52.19 52.52 52.65 52.85 53.02 53.22
 52.74 52.62 52.43 52.75 52.51 52.73 53.56 53.54 54.25 54.27 54.08 56.38
 56.4  56.76 56.59 56.6  56.28 55.96 55.74 55.28 55.68 55.59 54.77 55.5
 55.83 56.16 57.87 58.08 58.06 58.63 58.71 59.18 57.92 57.46 57.31 57.01
 57.5  57.56 58.55 58.74 59.59 59.29 59.21 59.43 59.48 59.62 60.02 59.89
 58.95 58.72 58.59 58.85 58.62 58.35 58.52 57.53 57.63 58.52 58.98 59.85
 59.98 59.61 59.19 59.2  59.1  58.86 58.77 58.93 59.

In [62]:
# Preprocess approach 2
prices = np.loadtxt("../data/csco_2025.csv", delimiter=",", skiprows=1, usecols=1)

S=prices[::-1]
n=np.size(S)
delta_t= 1/n

start = n//2
i=np.arange(0, start)
X=np.log(S[i+1]/S[i])
sigma = np.std(X)/np.sqrt(delta_t)
mu = np.mean(X)/delta_t + sigma**2/2

mu, sigma

(np.float64(0.3812894070673739), np.float64(0.2019659529136761))

In [64]:
from Helpers.BSFormulas import delta_Put, Put

D = 0
S0=S[start]
E=S0
T=1/2 # Half a year
r = 0.03
price_put = Put(S0, E, T, r, sigma, D)
print("Put price:", price_put)
eta = delta_Put(S0, E, T, r, sigma, D)
print("eta:", eta)

Put price: 2.894458884706488
eta: -0.42997441229533984


In [65]:
# Performing trade simulation | European option

# self-financing portfolio
portfolio_value = np.zeros(n-start)
portfolio_value[0] = price_put

T_values = np.linspace(T, 0, n-start) # how much time is left until maturity
dt = T/(n-1-start) # delta time

current_portfolio_value = price_put

for t in range(start, n-1):
    # calc, how much time is until the expiration
    T_current = T_values[t-start]

    # How many shares we should hold in our portfolio
    eta = delta_Put(S[t], E, T_current, r, sigma, D)  # Key factor, we need to compute it at each time step

    # Compute the portfolio value at the next time step (next day)
    current_portfolio_value = current_portfolio_value + (current_portfolio_value-eta*S[t])*r*dt + D*eta*S[t]*dt + eta*(S[t+1]-S[t])

    # Updating the portfolio value
    portfolio_value[t-start+1] = current_portfolio_value

payoff_values = np.maximum(E-S[start:], 0)

In [66]:
print("Portfolio values:", portfolio_value) # Loss

Portfolio values: [ 2.89445888  2.86346059  2.66638482  3.17966157  3.39718893  3.47419152
  3.62684392  3.38014049  3.35468512  2.87820028  2.79871501  2.43875636
  2.55789202  2.59317766  2.50723958  2.49064292  2.4388241   2.28758384
  2.33819823  2.69171441  2.7914603   2.85091726  2.74043871  2.84156237
  2.96306761  2.88980939  3.33203474  3.28620228  2.85207911  2.65123161
  2.28920164  2.24347205  2.38261486  2.54830096  2.5474463   2.59159514
  2.69440138  2.73630573  2.67037408  2.55905224  2.7508322   2.74554327
  2.50093983  2.24220653  2.30385933  2.15283427  1.87320607  1.68936327
  1.52357982  1.52565415  2.31596205  2.17274124  2.12812853  1.77176586
  1.73058131  1.5881119   1.49902276  1.14655169  1.21646251  1.21841625
  1.09142999  1.17390244  1.15288913  0.8627375   0.69405617  0.73068813
  0.69815602  0.72043113  0.81036872  0.92402388  0.73956204  0.76687229
  0.8066569   0.76245342  0.85063828  0.88750319  0.77050499  0.84760306
  0.76483961  1.01930955  1.32897

In [67]:
print("Payoff values:", payoff_values)

Payoff values: [0.   0.   0.   0.71 1.17 1.32 1.62 1.13 1.07 0.08 0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.04 0.   0.01 0.28 0.11 1.1  1.
 0.11 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.32
 4.09 4.22 5.44 0.5  2.04 1.3  1.22 1.43 2.87 2.87 4.26 3.59 3.   2.34
 1.92 1.79 1.29 0.9  0.51 0.   0.   0.   0.   0.   0.   0.   0.  ]


In [57]:
# American put: Need to use solver for computing option price and values of the derivative at different time moments
# Good approach should be to use solver once to compute price of the option and all derivatives.